# Analysis of Experiment 1 and Strategy Evolution

## 1. Current Status
- **Best CV**: 121.43 (Baseline)
- **Target**: 68.92
- **Gap**: ~52.5 points
- **Approach**: C++ Optimization (SA + BP)
- **Observation**: The baseline run (5000 iters) worked and improved the score from ~173 to 121. This confirms the pipeline is valid.

## 2. Analysis of Top Kernels
I have analyzed `yongsukprasertsuk_santa-2025-best-keeping-bbox3-runner` (which seems to be a derivative of `smartmanoj`).
Key insights from this kernel:
- **Iterative Improvement**: It runs a loop of `bbox3` (which is likely the C++ optimizer) -> `fix_direction` (rotation optimization) -> `repair_overlaps`.
- **Time Management**: It has a "budget" (3 hours) and allocates time dynamically to promising `(n, r)` configurations.
- **Parameters**: It explores `n` (iterations?) in [1000, 2000] and `r` (restarts?) in [30, 90]. These are still relatively low numbers compared to what might be needed for the final target.
- **Rotation Optimization**: It uses `scipy.optimize.minimize_scalar` to fine-tune the rotation of the convex hull of the trees. This is a crucial step that my baseline missed (my baseline only relies on the C++ optimizer's internal rotation moves).
- **Overlap Repair**: It has a specific step to repair overlaps, which suggests the C++ optimizer might leave some or the rotation step might introduce them.

## 3. Missing Pieces in My Baseline
1. **Rotation Optimization**: The C++ code does some rotation, but a dedicated Python-based fine-tuning step (using `scipy` or `shapely`) seems to be standard in top kernels.
2. **Scale**: 5000 iterations is too low. I need to run for much longer.
3. **Loop Structure**: The "Pack -> BP -> Pack" loop is good, but "Pack -> Rotate -> BP -> Pack" might be better.
4. **Ensembling**: Running multiple seeds and picking the best `N` configuration is a cheap way to gain points.

## 4. Plan for Next Experiment
I will implement a more robust optimization loop that mimics the structure of the top kernels but with my own control.
- **Increase Compute**: Run `tree_packer` with `n=100,000` and `r=32`.
- **Add Rotation Step**: Implement the `optimize_rotation` function from the analyzed kernel.
- **Iterative Loop**: Run `tree_packer` -> `optimize_rotation` -> `bp` -> `tree_packer` ...
- **Parallelism**: The C++ code uses OpenMP, so I should ensure I'm using all cores.

I will also look at `smartmanoj`'s kernel again to see if I missed any other C++ files or parameters. The `yongsukprasertsuk` kernel refers to `bbox3`, which might be a compiled binary of `tree_packer`.

## 5. Hypothesis
Increasing iterations to 100k and adding a rotation post-processing step will significantly drop the score, likely below 100.
